In [2]:
pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 35.5 MB/s eta 0:00:0000:0100:01
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 52.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.8 MB/s eta 0:00:0000:0100:01
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 2.7 MB/s eta 0:00:0000:0100:01m
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181

In [11]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install --upgrade pip

  Using cached pip-23.2.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json
import shutil
import random
import numpy as np
from PIL import ImageFile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [18]:
def load_dataset_from_folders(image_folder, json_folder, num_samples=10):
    image_paths = []
    labels = []

    for food_name in os.listdir(image_folder):
        if food_name.startswith('.'):
            continue
        
        food_image_folder = os.path.join(image_folder, food_name)
        food_json_folder = os.path.join(json_folder, food_name)
        
        image_files = [file_name for file_name in os.listdir(food_image_folder) if file_name.endswith('.jpg')]

        sampled_images = random.sample(image_files, min(num_samples, len(image_files)))

        for file_name in sampled_images:
            image_path = os.path.join(food_image_folder, file_name)
            image_name = os.path.splitext(file_name)[0]

            json_file_path = os.path.join(food_json_folder + ' json/', image_name + '.json')

            with open(json_file_path, 'r') as f:
                annotation = json.load(f)

            label = annotation[0]['Name']
            label = label.replace('-', '')
            
            if label is not None:
                if label == 'bogsunga':
                    labels.append('peach')
                elif label == 'gamjaseupeu':
                    labels.append('potato_soup')
                else:
                    labels.append(label)
            image_paths.append(image_path)

    return image_paths, labels

In [19]:
image_folder = './Training/[원천]음식203_Tra/'
json_folder = './Training/[라벨]음식203_Tra_json/'
# image_folder = './fooddata/source/'
# json_folder = './fooddata/label/'

image_paths, labels = load_dataset_from_folders(image_folder, json_folder)

In [20]:
unique_labels = np.unique(labels)
unique_labels

array(['baeghyanggwa', 'beoseos_jangajji', 'bolippang', 'bossam', 'peach'],
      dtype='<U16')

In [69]:
label_mapping = {'baeghyang-gwa': '백향과', 'beoseos_jang-ajji': '버섯 장아찌', 'bolippang': '보리빵', 'bossam': '보쌈', 'peach': '복숭아'}
# label_mapping = {'galbigu-i': '갈비구이', 'gamjasaelleodeu': '감자샐러드', 'ganjang': '간장', 'potato_soup': '감자스프',
#                  'gamjagu-i': '감자구이','gamjabokk-eum':'감자볶음','gam':'감','tangerine_juice':'감귤주스','gacheudong':'가츠동',
#                  'gaji':'가지구이', 'gamjageulatang':'감자그라탕'}

korean_labels = [label_mapping[label] for label in labels]

unique_korean_labels = np.unique(korean_labels)
label_to_int = {label: i for i, label in enumerate(unique_korean_labels)}
int_labels = np.array([label_to_int[label] for label in korean_labels])

In [70]:
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(image_paths, int_labels, test_size=0.2, random_state=42)

In [71]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(unique_korean_labels), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [32]:
batch_size = 32
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    preprocessing_function = preprocess_input
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

In [36]:
# colab용
# train_dir = '/content/drive/MyDrive/final/train'
# val_dir = '/content/drive/MyDrive/final/val'
# jupyter용
train_dir = './fooddata/final/train'
val_dir = './fooddata/final/val'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Copy train images to the train directory
for image_path in train_image_paths:
    directory_path = os.path.dirname(image_path)
    class_name = os.path.basename(directory_path)
    destination_dir = os.path.join(train_dir, class_name)
    os.makedirs(destination_dir, exist_ok=True)
    shutil.copy(image_path, destination_dir)

# Copy validation images to the validation directory
for image_path in val_image_paths:
    directory_path = os.path.dirname(image_path)
    class_name = os.path.basename(directory_path)
    destination_dir = os.path.join(val_dir, class_name)
    os.makedirs(destination_dir, exist_ok=True)
    shutil.copy(image_path, destination_dir)

In [37]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (299, 299),
    batch_size = batch_size,
    class_mode = 'sparse',
    classes = unique_korean_labels.tolist(),
    shuffle = True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = (299, 299),
    batch_size = batch_size,
    class_mode = 'sparse',
    classes = unique_korean_labels.tolist()
)

epochs = 10

Found 220 images belonging to 5 classes.
Found 44 images belonging to 5 classes.


In [38]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
model.fit(train_generator, validation_data=val_generator, epochs=epochs)

NameError: name 'scipy' is not defined

In [ ]:
loss, accuracy = model.evaluate(val_generator)
print(f"Validation loss: {loss:.4f}, Validation accuracy: {accuracy:.4f}")

In [ ]:
model.save('./food_recognition_model.h5')

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/food_recognition_model_js.h5')

In [ ]:
# Step 9: Make Predictions on New Images
def predict_food_category(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.expand_dims(img, axis=0)
    img = preprocess_input(img)

    keys_with_value = []

    prediction = model.predict(img)[0]
    predicted_class_index = np.argmax(prediction)
    for key, value in label_to_int.items():
        if value == predicted_class_index:
            keys_with_value.append(key)
    return keys_with_value

In [ ]:
# Replace 'path/to/image.jpg' with the path to the image you want to test
input_image_path = './food_test/1.jpg'
predicted_food = predict_food_category(input_image_path)
print("Predicted Food Category:", predicted_food)